Install all required packages and libraries

In [53]:
!pip install ultralytics torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install opencv-python matplotlib seaborn pandas numpy Pillow
!pip install roboflow supervision
!pip install colorama

Looking in indexes: https://download.pytorch.org/whl/cu118


Get YOLO detector

In [54]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt
%cd ..

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5
/content


Load datasets

In [63]:
import os

from pathlib import Path

from google.colab import drive

drive.mount('/content/drive')

SHARED_PATH = Path("drive/MyDrive/Colab Notebooks/Shared")
HRSC_PATH = SHARED_PATH / "HRSC2016_Final_Splits"
DOTA_PATH = SHARED_PATH / "DOTA_Final_Splits"

print("HRSC important subfolders:", [f"{subfolder.name}/{path.name}" for subfolder in HRSC_PATH.iterdir() for path in subfolder.iterdir()])
print("DOTA important subfolders:", [f"{subfolder.name}/{path.name}" for subfolder in DOTA_PATH.iterdir() for path in subfolder.iterdir()])

HRSC_TRAIN_IMAGES = HRSC_PATH/ "train/images"
HRSC_TRAIN_ANNOTATIONS = HRSC_PATH / "train/annotations"
HRSC_VAL_IMAGES = HRSC_PATH / "val/images"
HRSC_VAL_ANNOTATIONS = HRSC_PATH / "val/annotations"
HRSC_TEST_IMAGES = HRSC_PATH / "test/images"
HRSC_TEST_ANNOTATIONS = HRSC_PATH / "test/annotations"

DOTA_TRAIN_IMAGES = DOTA_PATH / "train/images"
DOTA_TRAIN_ANNOTATIONS = DOTA_PATH / "train/hbb"
DOTA_VAL_IMAGES = DOTA_PATH / "val/images"
DOTA_VAL_ANNOTATIONS = DOTA_PATH / "val/hbb"
DOTA_TEST_IMAGES = DOTA_PATH / "test/images"
DOTA_TEST_ANNOTATIONS = DOTA_PATH / "test/hbb"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
HRSC important subfolders: ['train/images', 'train/annotations', 'val/images', 'val/annotations', 'test/images', 'test/annotations']
DOTA important subfolders: ['train/images', 'train/hbb', 'val/images', 'val/hbb', 'test/images']


Explore datasets

In [64]:
from colorama import Fore, Style

def explore_header(dataset: str, subfolder: str):
  print("Exploring", Fore.GREEN + dataset + Style.RESET_ALL, Fore.MAGENTA + subfolder + Style.RESET_ALL, "folder...")

def explore(images_folder: Path, ext: str):
  files = list(images_folder.glob(f"*.{ext}"))
  print(f"Number of {ext.upper()} files:", len(files))
  print(f"{ext.upper()} sample files:", [path.name for path in files[:3]])

explore_header("HRSC", "train")
explore(HRSC_TRAIN_IMAGES, "bmp")
explore(HRSC_TRAIN_ANNOTATIONS, "xml")

print()

explore_header("HRSC", "val")
explore(HRSC_VAL_IMAGES, "bmp")
explore(HRSC_VAL_ANNOTATIONS, "xml")

print()

explore_header("HRSC", "test")
explore(HRSC_TEST_IMAGES, "bmp")
explore(HRSC_TEST_ANNOTATIONS, "xml")

print()

explore_header("DOTA", "train")
explore(DOTA_TRAIN_IMAGES, "png")
explore(DOTA_TRAIN_ANNOTATIONS, "txt")

print()

explore_header("DOTA", "val")
explore(DOTA_VAL_IMAGES, "png")
explore(DOTA_VAL_ANNOTATIONS, "txt")

print()

explore_header("DOTA", "test")
explore(DOTA_TEST_IMAGES, "png")
explore(DOTA_TEST_ANNOTATIONS, "txt")


Exploring HRSC train folder...
Number of BMP files: 436
BMP sample files: ['100000001.bmp', '100000002.bmp', '100000004.bmp']
Number of XML files: 436
XML sample files: ['100000001.xml', '100000002.xml', '100000004.xml']

Exploring HRSC val folder...
Number of BMP files: 181
BMP sample files: ['100000006.bmp', '100000010.bmp', '100000622.bmp']
Number of XML files: 181
XML sample files: ['100000006.xml', '100000010.xml', '100000622.xml']

Exploring HRSC test folder...
Number of BMP files: 453
BMP sample files: ['100000003.bmp', '100000005.bmp', '100000623.bmp']
Number of XML files: 453
XML sample files: ['100000003.xml', '100000005.xml', '100000623.xml']

Exploring DOTA train folder...
Number of PNG files: 1411
PNG sample files: ['P0805.png', 'P0806.png', 'P0809.png']
Number of TXT files: 1411
TXT sample files: ['P0805.txt', 'P0806.txt', 'P0809.txt']

Exploring DOTA val folder...
Number of PNG files: 458
PNG sample files: ['P0003.png', 'P0004.png', 'P0007.png']
Number of TXT files: 458


Match images with annotations

In [68]:
class Dataset:
  def __init__(self, images_folder: Path, image_ext: str, annotations_folder: Path, annotation_ext: str):
    self.images = list(images_folder.glob(image_ext))
    self.annotations = list(annotations_folder.glob(annotation_ext))

    image_ids = set(path.stem for path in self.images)
    annotation_ids = set(path.stem for path in self.annotations)
    self.ids = list(set(image_ids) & set(annotation_ids))
    self.images = [path for path in self.images if path.stem in self.ids]
    self.annotations = [path for path in self.annotations if path.stem in self.ids]

HRSC_TRAIN_DATASET = Dataset(HRSC_TRAIN_IMAGES, "*.bmp", HRSC_TRAIN_ANNOTATIONS, "*.xml")
print("HRSC train dataset sample:", HRSC_TRAIN_DATASET.ids[:5])
HRSC_VAL_DATASET = Dataset(HRSC_VAL_IMAGES, "*.bmp", HRSC_VAL_ANNOTATIONS, "*.xml")
print("HRSC val dataset sample:", HRSC_VAL_DATASET.ids[:5])
HRSC_TEST_DATASET = Dataset(HRSC_TEST_IMAGES, "*.bmp", HRSC_TEST_ANNOTATIONS, "*.xml")
print("HRSC test dataset sample:", HRSC_TEST_DATASET.ids[:5])

DOTA_TRAIN_DATASET = Dataset(DOTA_TRAIN_IMAGES, "*.png", DOTA_TRAIN_ANNOTATIONS, "*.txt")
print("DOTA train dataset sample:", DOTA_TRAIN_DATASET.ids[:5])
DOTA_VAL_DATASET = Dataset(DOTA_VAL_IMAGES, "*.png", DOTA_VAL_ANNOTATIONS, "*.txt")
print("DOTA val dataset sample:", DOTA_VAL_DATASET.ids[:5])
DOTA_TEST_DATASET = Dataset(DOTA_TEST_IMAGES, "*.png", DOTA_TEST_ANNOTATIONS, "*.txt")
print("DOTA test dataset sample:", DOTA_TEST_DATASET.ids[:5])


HRSC train dataset sample: ['100000860', '100000770', '100000852', '100000670', '100000886']
HRSC val dataset sample: ['100000717', '100001115', '100001438', '100000905', '100000715']
HRSC test dataset sample: ['100001591', '100000710', '100001417', '100001206', '100001309']
DOTA train dataset sample: ['P2145', 'P2055', 'P1098', 'P1121', 'P2090']
DOTA val dataset sample: ['P1210', 'P2721', 'P0228', 'P1492', 'P0704']
DOTA test dataset sample: []
